# Initialization

In [1]:
#
# Imports
#

import requests
import io
from astropy.table import Table

# There are a number of relatively unimportant warnings that 
# show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

# 
# Convenience functions
# 

def aptable_from_votable_response(response):
    """
    Takes a VOTABLE response from a web service and returns an astropy table.
    
    Parameters
    ----------
    response : requests.Response
        Response whose contents are assumed to be a VOTABLE.
        
    Returns
    -------
    aptable : astropy.table.Table
        Astropy Table containing the data from the first TABLE in the VOTABLE.
    """
    
    # The astropy table reader would like a file-like object, so convert
    # the response content a byte stream.  This assumes Python 3.x.
    # 
    # (The reader also accepts just a string, but that seems to have two 
    # problems:  It looks for newlines to see if the string is itself a table,
    # and we need to support unicode content.)
    file_like_content = io.BytesIO(response.content)
    
    # The astropy table reader will auto-detect that the content is a VOTABLE
    # and parse it appropriately.
    aptable = Table.read(file_like_content)
    return aptable


# Examples
## TAP Query from HEASARC
Here, we will use a HEASARC catalog called 'zcat' that has columns including RA, DEC, and Bmag. The selection criteria match a single source.  

In [2]:
tap_params = {
    "request":"doQuery",
    "lang":"ADQL",
    "query":
    """SELECT * FROM zcat as cat where 
    cat.RA > 191. and cat.RA < 192 and cat.Dec > 30. and cat.Dec < 31 and cat.Bmag > 14.9"""
    }
response = requests.get('https://heasarc.gsfc.nasa.gov/xamin/vo/tap/sync', params=tap_params)

# The aptable_from_votable_response call below is just an overly documented
# wrapper for these two lines of code which do the same thing:
##
##    file_like_content = io.BytesIO(response.content)
##    aptable = Table.read(file_like_content)

aptable = aptable_from_votable_response(response)

aptable


__row,name,ra,dec,lii,bii,bmag,radial_velocity,radial_velocity_error,ref_bmag,ref_radial_velocity,morph_type,bar_type,luminosity_class,structure,diameter_1,diameter_2,bt_mag,ugc_or_eso,distance,rfn_number,comments,redshift,ref_redshift,notes,class,__x_ra_dec,__y_ra_dec,__z_ra_dec
int32,object,float64,float64,float64,float64,float32,int32,int32,object,int32,int32,object,int32,object,float64,float64,float64,object,float64,object,object,float64,int32,object,int32,float64,float64,float64
44890,N4676A,191.54198643000001,30.732265689999998,140.35832761,86.216130669999998,15.0,6651,50,H,2712,20,,-1,P,2.2000000000000002,0.34999999999999998,--,07938,--,,"VV224,I 819,Arp242",--,-1,,6200,-0.17198682305215199,-0.84218280700419401,0.51102705631205503


## Finding SkyView SIA resources from the NAVO directory with TAP

In [3]:
tap_params["query"]="""
    select b.short_name,a.ivoid,a.cap_type from rr.capability a 
    natural join rr.resource b 
    where a.cap_type='simpleimageaccess' and a.ivoid like 'ivo://nasa.heasarc%' 
    order by short_name;
    """
r = requests.post('http://vao.stsci.edu/RegTAP/TapService.aspx/sync', data=tap_params)

aptable = aptable_from_votable_response(r)
aptable


short_name,ivoid,cap_type
object,object,object
1420MHz,ivo://nasa.heasarc/skyview/1420mhz,simpleimageaccess
2MASS,ivo://nasa.heasarc/skyview/2mass,simpleimageaccess
408MHz,ivo://nasa.heasarc/skyview/408mhz,simpleimageaccess
AKARI,ivo://nasa.heasarc/skyview/akari,simpleimageaccess
ASCA,ivo://nasa.heasarc/ascamaster,simpleimageaccess
ASCA/AGN,ivo://nasa.heasarc/tartarus,simpleimageaccess
ASCASIS,ivo://nasa.heasarc/ascasis,simpleimageaccess
BATSIG,ivo://nasa.heasarc/skyview/batsig,simpleimageaccess
CDFS-LESS,ivo://nasa.heasarc/skyview/cdfs-less,simpleimageaccess


## HLA SIA search around M61

In [4]:
params = {
    'POS': '185.47873,4.47365',
    'SIZE': '0.001'
}
r = requests.get('http://hla.stsci.edu/cgi-bin/hlaSIAP.cgi?imagetype=best&inst=ACS,ACSGrism,WFC3,WFPC2,NICMOS,NICGRISM,COS,STIS,FOS,GHRS&proprietary=false&',
                params=params)

aptable = aptable_from_votable_response(r)
aptable['Dataset','URL']

Dataset,URL
object,object
hst_05446_4d_wfpc2_f606w_wf,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_05446_4d_wfpc2_f606w_wf
hst_05446_4d_wfpc2_total_wf,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_05446_4d_wfpc2_total_wf
hst_06358_08_wfpc2_f218w_wf,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_06358_08_wfpc2_f218w_wf
hst_06358_08_wfpc2_total_wf,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_06358_08_wfpc2_total_wf
hst_09042_62_wfpc2_f450w_wf,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_09042_62_wfpc2_f450w_wf
hst_09042_62_wfpc2_f814w_wf,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_09042_62_wfpc2_f814w_wf
hst_09042_62_wfpc2_total_wf,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_09042_62_wfpc2_total_wf
hst_09042_62_wfpc2_f814w_f450w_wf,http://hla.stsci.edu/cgi-bin/fitscut.cgi?red=hst_09042_62_wfpc2_f814w_wf&amp;blue=hst_09042_62_wfpc2_f450w_wf&amp;size=ALL&amp;format=fits&amp;config=ops
hst_12574_01_acs_wfc_f435w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_12574_01_acs_wfc_f435w


## UCD Handling
### Show the UCD for each Column

In [27]:
# If the VOTABLE FIELD has a UCD, it will be present in the metadata (key=='ucd')  
# for the corresponding Column in the astropy Table.
name_ucd = []
for key in aptable.columns:
    col = aptable.columns[key]
    name = col.name
    ucd = col.meta.get('ucd')
    name_ucd.append([str(name), str(ucd)])

name_ucd_table = Table(rows=name_ucd, names=('Column Name', 'UCD'))
name_ucd_table

Column Name,UCD
str14,str25
URL,VOX:Image_AccessReference
RA,POS_EQ_RA_MAIN
DEC,POS_EQ_DEC_MAIN
Level,PRODUCT_LEVEL
Target,TARGET_NAME
Detector,INST_ID
Aperture,INST_APERT
Spectral_Elt,MAIN_FILTER
NExposures,NUM_EXPOSURES


### Find title image access URL columns by UCD from an SIA response

In [38]:
#
# Define function to find a column by UCD.
#
def find_column_by_ucd(table, ucd):
    
    # Loop through all the columns looking for the UCD
    for key in table.columns:
        col = table.columns[key]
        ucdval = col.meta.get('ucd')
        if (ucdval is not None):
            if (ucd == ucdval):
                return col
    
    return None

#
# Perform an SIA query.
#
params = {
    'POS': '185.47873,4.47365',
    'SIZE': '0.001'
}
r = requests.get('http://hla.stsci.edu/cgi-bin/hlaSIAP.cgi?imagetype=best&inst=ACS,ACSGrism,WFC3,WFPC2,NICMOS,NICGRISM,COS,STIS,FOS,GHRS&proprietary=false&',
                params=params)
aptable = aptable_from_votable_response(r)

#
# Use UCDs to find the title and access URL columns, then display the results with just those columns.
#
title_col = find_column_by_ucd(aptable, 'VOX:Image_Title')
access_url_col = find_column_by_ucd(aptable, 'VOX:Image_AccessReference')

#
# Output the results if successful.
#
if (title_col is not None and access_url_col is not None):
    result = aptable[title_col.name, access_url_col.name]
else:
        result = 'A needed column was not found.  title_col = ' + str(title_col) + ', access_url_col = ' + str(access_url_col)

result


Title,URL
object,object
05446_4d WFPC2 F606W (combined) NGC4303,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_05446_4d_wfpc2_f606w_wf
05446_4d WFPC2 detection (combined) NGC4303,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_05446_4d_wfpc2_total_wf
06358_08 WFPC2 F218W (combined) NGC4303,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_06358_08_wfpc2_f218w_wf
06358_08 WFPC2 detection (combined) NGC4303,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_06358_08_wfpc2_total_wf
09042_62 WFPC2 F450W (combined) NGC4303,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_09042_62_wfpc2_f450w_wf
09042_62 WFPC2 F814W (combined) NGC4303,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_09042_62_wfpc2_f814w_wf
09042_62 WFPC2 detection (combined) NGC4303,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_09042_62_wfpc2_total_wf
09042_62 WFPC2 F814W/F450W (color) NGC4303,http://hla.stsci.edu/cgi-bin/fitscut.cgi?red=hst_09042_62_wfpc2_f814w_wf&amp;blue=hst_09042_62_wfpc2_f450w_wf&amp;size=ALL&amp;format=fits&amp;config=ops
12574_01 ACS/WFC F435W (combined) SN-2006OV,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_12574_01_acs_wfc_f435w
